# Optymalizacje

## rozgrzewka - Tips and Tricks

Trochę sztuczek i tricków. Niektóre mogą być przydatne przy optymalizacji. Potraktujmy to jako formę rozgrzewki

In [303]:
import dis

#### bardziej zwięzłe wyrażenia warynkowe przypisujące wartości

In [296]:
condition = True

In [301]:
%%timeit
if condition:
    x = 1
else:
    x = 0

30.4 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [302]:
%%timeit
x = 1 if condition else 0

31.5 ns ± 0.671 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [299]:
dis.dis("""if condition:
    x = 1
else:
    x = 0""")

  1           0 LOAD_NAME                0 (condition)
              2 POP_JUMP_IF_FALSE       10

  2           4 LOAD_CONST               0 (1)
              6 STORE_NAME               1 (x)
              8 JUMP_FORWARD             4 (to 14)

  4     >>   10 LOAD_CONST               1 (0)
             12 STORE_NAME               1 (x)
        >>   14 LOAD_CONST               2 (None)
             16 RETURN_VALUE


In [300]:
dis.dis("""x = 1 if condition else 0""")

  1           0 LOAD_NAME                0 (condition)
              2 POP_JUMP_IF_FALSE        8
              4 LOAD_CONST               0 (1)
              6 JUMP_FORWARD             2 (to 10)
        >>    8 LOAD_CONST               1 (0)
        >>   10 STORE_NAME               1 (x)
             12 LOAD_CONST               2 (None)
             14 RETURN_VALUE


#### Separator _ w liczbach

In [311]:
num1 = 10000000000
num2 = 10000000

print(num1 + num2)

10010000000


In [320]:
num1 = 10_000_000_000
num2 = 10_000_000

print(f'{num1 + num2:_}')

10_010_000_000


#### iterowanie po kilku listach

In [322]:
names = ['Peter Parker', 'Clark Kent', 'Wade Wilson', 'Bruce Wayne']
heroes = ["Spiderman", "Superman", "Deadpool", "Batman"]

In [324]:
for i, name in enumerate(names):
    hero = heroes[i]
    print(f"{name} to {hero}")

Peter Parker to Spiderman
Clark Kent to Superman
Wade Wilson to Deadpool
Bruce Wayne to Batman


In [325]:
for name, hero in zip(names, heroes):
    print(f"{name} to {hero}")

Peter Parker to Spiderman
Clark Kent to Superman
Wade Wilson to Deadpool
Bruce Wayne to Batman


In [326]:
universes = ['Marvel', 'DC', 'Marvel', 'DC']

for name, hero, universe in zip(names, heroes, universes):
    print(f"{name} to {hero} from {universe}")

Peter Parker to Spiderman from Marvel
Clark Kent to Superman from DC
Wade Wilson to Deadpool from Marvel
Bruce Wayne to Batman from DC


#### Rozpakowywanie

In [329]:
a, b, *_, d = (1, 2, 3, 4, 5, 6, 7, 8)  # czy to zadziała i jak?

#### `type`, `hasattr`, `getattr`, `setattr`

In [333]:
class Person():
    pass

person = Person()

print(hasattr(person, 'first_name'))
setattr(person, 'first_name', 'Corey')
print(hasattr(person, 'first_name'))
print(person.first_name)

key = 'first_name'
value = getattr(person, key)
print(value)

False
True
Corey
Corey


In [351]:
x = type("Dynamiczny",(),{'ala': "kot"})
print(type(x))
print(type.__name__)
x.ala

y = x()
print(type(y))


<class 'type'>
type
<class '__main__.Dynamiczny'>


##### Zadanie

Korzystając z modułu `json`. 
Odczytaj plik musicians.json (jest w katalogu dane).

Utwórz dynamicznie klasy i ich instancje - na podstawie pola type
Ustaw instancjom atrybuty - bez atrybutu type
Dodaj instancję do listy `instances`



In [ ]:
#### hasło w konsoli

In [358]:
username = input("Username: ")
password = input("Password: ")

print("Logging in...")

Username: 123
Password: 123
Logging in...


In [356]:
with open("data/musicians.json") as f:
    dane = json.load(f)

instances = []
for el in dane:
    type_ = el.pop('type')
    tmp_obj = type(type_, (),{})()
    
    for k, v in el.items():
        setattr(tmp_obj, k, v)
    instances.append(tmp_obj)
instances

In [359]:
from getpass import getpass

username = input("Username: ")
password = getpass("Password: ")

print("Logging in...")

Username: 123
Password: ········
Logging in...


#### generatory

Problem - chcemy policzyć linie w pliku, ale jest na tyle duzy, że nie chcemy otworzyć go w całości i robić splitlines, a zrobić to w jakiś sprytniejszy sposób



In [360]:
with open('data/movies.csv') as f:
    print(sum([1 for line in f]))

313240


In [364]:
with open('data/movies.csv') as f:
    file_gen = (row for row in f)

print(type(file_gen))

<class 'generator'>


In [365]:
def file_gen(file_name):
    for row in open(file_name):
        yield row
        
wiersze = file_gen('data/moves.csv')
wiersze

<generator object file_gen at 0x7f23dd0269d0>

In [366]:
next(wiersze)

'Innocent Man\n'

In [367]:
next(wiersze)

'Innocent Man\n'

In [368]:
for i in range(10):
    print(next(wiersze))

Iron Cowboy: Story Of The 50-50-50

Iron Cowboy: Story Of The 50-50-50

John Wick / John Wick: Chapter 2

Jormungand / Jormungand: Perfect Order: The Complete Series

Just Friends

"Kakuriyo Bed & Breakfast For Spirits: Season 1, Part 1"

Kamisama Kiss: Season 2

"Khrustalyov, My Car!"

Kimagure Orange Road

Land Unknown



In [371]:
import sys
sys.getsizeof(wiersze)

128

##### Zadanie

Napisz generator, który będzie zwracać kolejne potęgi dwójki. 

In [380]:
def potegi_dwojki():
    i = 0
    while True:
        yield 2**i
        i += 1

c = potegi_dwojki()

for i in range(10):
    print(next(c))

1
2
4
8
16
32
64
128
256
512


## Czasowo złożoność obliczeniowa

Złożoność obliczeniową określamy jako funkcję danych wejściowych algorytmu. Wyznacza się ją licząc operacje potrzebne do jej wykoanania
W praktyce wystarczą oszacowania takiej funkcji. Wprowadza się tu notacja Ο (dużego O), notacja Ω (omega) i notacja Θ (theta).

Niektóre funkcje i określenia złożoności z nimi związane:

* log(n)- złożoność logarytmiczna
* n - złożoność liniowa
* nlog(n) - złożoność liniowo-logarytmiczna
* n<sup>2</sup> - złożoność kwadratowa
* n<sup>k</sup>  - złożoność wielomianowa
* 2<sup>n</sup>  - złożoność wykładnicza
* n! - złożoność wykładnicza, ponieważ n!>2n już od n=4


Przykład obrazujący wzrost czasu wykonania programu w zależności od jego czasowej założoności obliczeniowej

* Duże "O", np O(n<sup>2</sup>) - górne asymptoryczne oszacowanie złożoności czasowej

Przykład - algorytm sortowania bąbelkowego w najgorszym przypadku ma czas obliczeń rzędu O(n<sup>2</sup>). Nie oznacza to jednak, że za każdym razem czas będzie to taki czas. 
Jeśli np dane wejściowe będa posortowane, to czas obliczeń wyniesie zaledwie O(N) (nie mówimy tutaj o czasie w sensie czasu zegarowego, ale liczbie operacji dominujących). Tak więc notacja „O duże” informuje nas, że czas wykonywania tego algorytmu nie jest dłuży niż O(n<sup>2</sup>) - ale może być krótszy.


* Duże "Omega" - np. Ω(N) - odnosi się ograniczenia asymptotycznego tempa wzrostu czasu wykonywania algorytmu od dołu.

W tym przypadku chodzi o to, że np. algorytm sortowania bąbelkowego w najlepszym przypadku ma czas obliczeń rzędu Ω(N). Nie oznacza to jednak, że za każdym razem czas ten będzie taki sam. Jeżeli dana na wejściu tablica n elementowa nie będzie posortowana czas ten wyniesie Ω(n<sup>2</sup>). Tak więc notacja „Omega duże” informuje nas, że czas wykonywania danego algorytmu nie jest krótszy niż Ω(N) - ale może być dłuższy.

* Notacja „Theta” (przykładowo: Θ(N2)) - to oszacowanie pomiędzy O i Omega

Więcej informacji np. tutaj: https://www.samouczekprogramisty.pl/podstawy-zlozonosci-obliczeniowej/






## Reguły optymalizacj: 

(https://wiki.c2.com/?RulesOfOptimization)

Jeśli zamierzasz przystąpić do optymalizacji rozważ następujące reguły:

1. Nie rób tego

    Czy na pewno tego potrzebujesz?

2. Nie rób tego ... jeszcze.

    Jeśli potrzebujesz to:
    1. Skończ swój kod
    2. Napisz testy
    3. Ok.. teraz możesz optymalizować

3. Profiluj - przed optymalizacją
    
    Zanim zaczniesz coś zmieniać wykonaj profilowanie:
    1. cProfile
    2. pstats
    3. RunSnakeRun SnakeViz

### Poziomy optymalizacji:

Wychodzimy tu poza optymalizacje sprzętowe. 

#### Projekt (desing)

W pewnych sytuacjach korzystne, czy wręcz konieczne może być przepisanie projektu. 

Mogą to być:

* zmiana technologi - niektóre języki po prostu są szybsze od innych. Ale często też i trudniej się w nich pisze.
* zmiany w stosie technologicznym - może baza danych nie daje rady, może serwer jest za wolny? 
* architektury - np. przejście na rozwiązanie asynchroniczne, czy wieloprocesowe 

Tu prawdopodobnie można uzyskać najlepsze wyniki, ale jest to kosztowne. 
Trzeba dobrze wiedzieć czego się chce i co zastosować. 
Może się to opłacać dla krytcznych, często uruchamianych fragmentów systemu.

#### Algorytmy i struktury danych.

Stosując lepsze algorytmy możemy uzyskać znaczne przyspieszenie działania programu. Często trzeba tu nieźle pogłówkować, wpaśc na jakiś sprytny pomysł. Przydaje się wiedza z innych dziedzin - takich jak matematyka.

Przykład. Jeśli chcemy zsumować liczby z jakiegoś zakresu, to możemy użyć pętli for:
    

In [304]:
N = 500000

In [305]:
%%timeit -n2
suma = 0
for x in range(1, N+1):
    suma += x

33.9 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 2 loops each)


Sumowanie musimy wykonać N razy. Więcj jest to złożonośc O(n). Znajac lepszy algorytm możemy wykonać te działanie szybciej, np. możemy wykorzystać wzór na sume N elementów.

In [306]:
%timeit -n2 N * (1 + N) / 2

The slowest run took 7.90 times longer than the fastest. This could mean that an intermediate result is being cached.
440 ns ± 419 ns per loop (mean ± std. dev. of 7 runs, 2 loops each)


#### Optymalizacja kodu źródłowego

- poziom budowania aplikacji - ustawianie różnego rodzaju flag przy uruchamianiu, czy kompilowaniu kodu. Pozwala to na optymalizację kodu na poziomie maszyny
- optymalizacje na poziomie kompilacji - niektóre kompilatory mogą być szybsze od innych
- optymalizacje na poziomie środowiska uruchomieniowego -  tu może pomóc instalacja nowszej wersji oprogramowania
- No i wreszcie - wykorzystanie samego języka w sposób optymalny



#### Różne obszary optymalizacji

Optymalizować mozemy zarówno szybkość, pamięć, zajmowaną przestrzeń dyskową, dyskowe i sieciowe operacje I/O, zużycie energii ...

**<cite>Always code as if the guy who ends up maintaining yout code will be a violent psychopath who knows where your live</cite><br>
<small>John Woods</small>**

### Wybrane przykłady optymalizacji kodu źródłowego

#### Ile zajmuje zwykłe wywołanie funkcji i zwrócenie wartości?

In [13]:
def the_answer_to_life_universe_and_everything():
    return 42

In [14]:
%timeit -n10000 the_answer_to_life_universe_and_everything()

80.1 ns ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)




7.5 miliona lat to ... 236520000000000 sekund

In [15]:
sekundy = 7_500_000 * 365 * 24 * 60 * 60
sekundy

236520000000000

In [16]:
(sekundy * 10 **9) / 67.6

3.498816568047337e+21

Czyli ~ 3.5 x 10<sup>21</sup> szybciej niż w Autostopem przez Galaktykę ... :D. Nie jest źle.

https://www.youtube.com/watch?v=aboZctrHfK8&t

Dalej podobnie - będziemy pisać kod w różnych wersjach i porównywać ze sobą.. 

####  Zliczenie elementów w liście

In [17]:
MILION_ELEMENTS = [1]*1_000_000

In [18]:
%%timeit -n10

how_many = 0
for element in MILION_ELEMENTS:
    how_many += 1


38.3 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit -n10 len(MILION_ELEMENTS)

114 ns ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
ns = 1
um = 100
ms = 100_000
s = 100_000_000

In [21]:
37*ms / 107*ns

34579.43925233645

#### Wybieranie z listy

Problem: wybrać z listy określone elementy. Np. liczby parzyste.

In [22]:
MILLION_ELEMENTS = list(range(1000000))

In [23]:
%%timeit -n10
output = []
for element in MILLION_ELEMENTS:
    if element % 2:
        output.append(element)
            


65.4 ms ± 5.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit -n10 list(filter(lambda x: x % 2, MILLION_ELEMENTS))

128 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit -n10 [x for x in MILLION_ELEMENTS if x % 2]

56.7 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Pytać o zgode, czy prosic o przebaczenie

W niektórych przypadkach wydajniej jest spróbować coś zrobić i obsłużyć to jakoś w przypadku niepowodzenia. Np przy pomocy `try ... except`. 
Czasem jednak lepiej jest najpierw sprawdzić.


In [26]:
class Foo:
    a1 = "a"
    a2 = "b"
    a3 = "c"

foo = Foo()


##### Podejście `ask for permission`

In [27]:
%%timeit -n100
if hasattr(foo, 'a1'):
    foo.a1

249 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### Podejście `beg for forgiveness`

In [28]:
%%timeit -n100
try:
    foo.a1
except AttributeError:
    pass


41.1 ns ± 3.07 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


Jeszcze lepiej to widać przy większej ilości warunków

In [29]:
%%timeit -n100 

if (hasattr(foo, 'a1') and hasattr(foo, 'a2') and hasattr(foo, 'a3')):
    foo.a1
    foo.a2
    foo.a3

339 ns ± 8.21 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit -n100
try:
    foo.a1
    foo.a2
    foo.a3
except AttributeError:
    pass

196 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


Jeśli atrybutu jednak rzeczywiście nie ma to obsługa błędu staje się kosztowniejsza:

In [31]:
class Bar:
    pass


bar = Bar()

In [32]:
%%timeit -n100

if hasattr(bar, 'hello'):
    bar.hello

206 ns ± 5.26 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%%timeit -n100
try: 
    bar.hello
except AttributeError:
    pass


439 ns ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


* Jeśli spodziewamy się częstych problemów z atrybutami, to lepsze będzie podejście "ask for perrmission". 
* Jeśli takich sytuacji spodziewamy się rzadko, to przewagę da podejście "beg for forgiveness"

#### Sprawdzanie przynależności - membership

In [34]:
def check_membership(number):
    for item in MILLION_ELEMENTS:
        if item == number:
            return True
    return False



In [35]:
%timeit -n10 check_membership(1)

230 ns ± 56.8 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%timeit -n100 check_membership(500000)

20.4 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
%timeit -n100 check_membership(1000000)

29.8 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
def check_membership2(number):
    return number in MILLION_ELEMENTS

In [39]:
%timeit -n100 check_membership2(1)

272 ns ± 3.32 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
%timeit -n100 check_membership2(500000)

5.66 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Oba podejścia nadal przechodzą przez całą listę.** 

Jeśli szukamy liczby z początku listy to mamy szczeście, jeśli z końca to musimy poczekać.

Przydałaby się struktura danych ze stałym czasem dostępu. Takimi strukturami są np. `set` i `dict`

In [41]:
MILLION_SET = set(MILLION_ELEMENTS)

In [42]:
%timeit -n100 1 in MILLION_SET

72.5 ns ± 3.5 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%timeit -n100 999999 in MILLION_SET

51.9 ns ± 3.3 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Ale!! Zamiana listy na set jest kosztowna!**

Opłaca się to robić więc tylko wtedy, gdy takich sprawdzeń robimy dużo. Najpierw zamieniamy listę na set a potem sprawdzamy. 

In [44]:
%timeit -n2 set(MILLION_ELEMENTS)

50.9 ms ± 8.64 ms per loop (mean ± std. dev. of 7 runs, 2 loops each)


#### Usuwanie duplikatów

Trzeba tez pamiętać o tym, że w zbiorze nie ma powtórzeń! Zamiana na set jest najszybszym sposobem pozbycia sie duplikatów. Poniżej koszmarny alogrytm - złożoność O(n<sup>2</sup>):

In [50]:
ELEMENTS = list(range(1, 1001))

In [51]:
%%timeit -n2

unique = []
for element in ELEMENTS:
    if element not in unique:
        unique.append(element)

7.13 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 2 loops each)


I szybki sposób

In [53]:
%timeit -n2 set(ELEMENTS)

27.9 µs ± 3.55 µs per loop (mean ± std. dev. of 7 runs, 2 loops each)


#### Sortowanie list

In [54]:
import random

RANDOM_NUMBERS = [random.random() for i in range(1000000)]

In [57]:
%timeit -n3 sorted(RANDOM_NUMBERS)

71.3 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [58]:
%timeit -n3 RANDOM_NUMBERS.sort()

30.1 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


#### Wiele operacji z jedną funkcją.


In [60]:
def kwadrat(n):
    return n**2

In [70]:
%timeit -n10000 [kwadrat(i) for i in range(1000)]

331 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [71]:
def oblicz_kwadraty():
    return [i**2 for i in range(1000)]

In [72]:
%timeit -n10000 oblicz_kwadraty()

296 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Instrukcje warunkowe, testowanie prawdy i fałszu, sprawdzanie czy struktury są puste czy nie

In [73]:
v1 = True
v2 = False

In [74]:
%%timeit 
if v1 == True:
    pass

33.9 ns ± 0.375 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [75]:
%%timeit
if v2 == True:
    pass

34.8 ns ± 0.364 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [76]:
%%timeit
if v1 is True:
    pass

27.6 ns ± 0.378 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [77]:
%%timeit
if v2 is True:
    pass

30 ns ± 1.07 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [80]:
%%timeit
if v1:
    pass

20.7 ns ± 0.202 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [81]:
%%timeit
if v2:
    pass

21.9 ns ± 0.498 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [82]:
%%timeit
if not v2:
    pass

22.7 ns ± 0.294 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Podobnie możemy zoptymalizować czy struktury danych są puste

In [84]:
lista = []

In [85]:
%%timeit

if len(lista) == 0:
    pass

75.6 ns ± 2.15 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [86]:
%%timeit

if lista == []:
    pass

47.5 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [87]:
%%timeit

if not lista:
    pass

25.5 ns ± 0.332 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Czy jest różnica w szybkości między zwykłą funkcją a lambdą?


In [88]:
def hello1(name):
    return f"Hello {name}"

hello2 = lambda name: f"Hello {name}"

In [94]:
%timeit hello1("Rafał")

147 ns ± 3.12 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [95]:
%timeit hello2("Rafał")

147 ns ± 6.29 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [96]:
import dis
dis.dis(hello1)

  2           0 LOAD_CONST               1 ('Hello ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 BUILD_STRING             2
              8 RETURN_VALUE


In [97]:
dis.dis(hello2)

  4           0 LOAD_CONST               1 ('Hello ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 BUILD_STRING             2
              8 RETURN_VALUE


#### `list()` czy `[]`

In [98]:
%timeit list()

99.6 ns ± 0.486 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [99]:
%timeit []

22.2 ns ± 0.427 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [101]:
dis.dis('[]')

  1           0 BUILD_LIST               0
              2 RETURN_VALUE


In [102]:
dis.dis('list()')

  1           0 LOAD_NAME                0 (list)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


In [103]:
%timeit dict()

102 ns ± 1.65 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [104]:
%timeit {}

40.7 ns ± 0.293 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Sloty w klasach

Klasy w zasadzie oferują nam możliwość dynamicznego tworzenia atrybutów. Ma to z pewnością swoje zalety, ale może prowadzić do problemów z pamięcią. Słownik to dynamiczna struktura. Czy da się tak zdefiniować klasę, by możliwe było korzystanie z tylko określonych atrybutów? Tak. Służy do tego specjalny atrybut `__slots__`. 

In [118]:
class MyClass1(object):
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier

    def hello(self):
        print("Hello")

In [119]:
class MyClass2(object):
    __slots__ = ['name', 'identifier']

    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier

    def hello(self):
        print("Hello")

In [120]:
%timeit MyClass1("a", 1)

287 ns ± 5.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [121]:
%timeit MyClass2("a", 1)

228 ns ± 3.77 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [122]:
%timeit MyClass1('a', 1).name

302 ns ± 3.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [123]:
%timeit MyClass2('a', 1).name

245 ns ± 2.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


##### Zbadajmy zużycie pamięci

In [130]:
# !pip install ipython_memory_usage

In [132]:
import ipython_memory_usage.ipython_memory_usage as imu

In [133]:
imu.start_watching_memory()

In [133] used 0.0000 MiB RAM in 1.11s, peaked 0.00 MiB above current, total RAM usage 321.11 MiB


In [145]:
%%writefile slots.py
class MyClass(object):
        __slots__ = ['name', 'identifier']
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

num = 1024*256
x = [MyClass(1,1) for i in range(num)]

Writing slots.py
In [145] used 0.0039 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 338.43 MiB


In [146]:
%%writefile noslots.py
class MyClass(object):
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

num = 1024*256
x = [MyClass(1,1) for i in range(num)]

Writing noslots.py
In [146] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 338.43 MiB


In [147]:
import slots

In [147] used 13.1250 MiB RAM in 0.44s, peaked 0.00 MiB above current, total RAM usage 351.56 MiB


In [148]:
import noslots

In [148] used 45.8633 MiB RAM in 0.68s, peaked 0.00 MiB above current, total RAM usage 397.42 MiB


In [150]:
imu.stop_watching_memory()

#### Przykład. Łączenie napisów

In [1]:
text = [str(x) for x in range(600000)]

In [2]:
def for_concat(lista):
    x = ""
    for i in lista:
        x += i
    return x

In [3]:
%timeit -n10 x = for_concat(text)


61.1 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
timeit -n10 x = "".join(text)

9.35 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
my_var = "my"
my_var1 = "1"
my_var2 = "3 4"


In [6]:
%timeit -n1000000 msg = 'hello ' + my_var + ' world' + my_var1 + my_var2

222 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%timeit -n1000000 msg = 'hello %s world %s %s' % (my_var, my_var1, my_var2)

282 ns ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:

%timeit -n1000000 msg = 'hello {} world {} {}'.format(my_var, my_var1, my_var2) 

327 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%timeit -n1000000 msg = f'hello {my_var} world {my_var1} {my_var2}'

154 ns ± 8.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


#### Zliczanie linii w pliku - różne optymalizacje

In [384]:
import mmap
import numpy as np
import pandas as pd
import time
import mmap
import random
from collections import defaultdict


filename = "data/PESEL_NAZWISKA.csv"  # 298127 linii


def simplecount(filename):
    lines = 0
    with open(filename) as f:
        for line in f:
            lines += 1
    return lines


def simplecount2(filename):
    with open(filename) as f:
        lines = sum(1 for line in f)
    return lines


def enumerate_count(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


def mapcount(filename):
    lines = 0
    with open(filename) as f:
        buf = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)
        readline = buf.readline
        while readline():
            lines += 1
    return lines


def bufcount(filename):
    lines = 0
    buf_size = 1024 * 1024

    with open(filename) as f:
        lines = 0
        read_f = f.read  # loop optimization
        buf = read_f(buf_size)
        while buf:
            lines += buf.count('\n')
            buf = read_f(buf_size)
    return lines


def numpy_count(filename):
    my_data = np.genfromtxt(filename, delimiter=',')
    return my_data.shape[0]


def pd_read(filename):
    df = pd.read_csv(filename)
    return df.shape[0] + 1


methods = [
    simplecount,
    simplecount2,
    enumerate_count,
    mapcount,
    bufcount,
    numpy_count,
    pd_read,


]

counts = defaultdict(list)

for i in range(5):
    for func in methods:
        start_time = time.time()
        assert func("data/PESEL_NAZWISKA.csv") == 298127
        counts[func].append(time.time() - start_time)

for key, vals in counts.items():
    print(key.__name__, ":", sum(vals) / float(len(vals)))

simplecount : 0.06078343391418457
simplecount2 : 0.05724267959594727
enumerate_count : 0.060010099411010744
mapcount : 0.029202699661254883
bufcount : 0.021102523803710936
numpy_count : 1.140993881225586
pd_read : 0.20309219360351563


#### Optymalizacje, które trzeba stosować ze szczególną uwagą:

##### przypisywanie wartości do zmiennych

In [154]:
%%timeit
q=1
w=1
e=1
r=1
t=1
y=1
u=1
i=1
o=1
p=1

53.8 ns ± 0.59 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [155]:
%%timeit
q,w,e,r,t,y,u,i,o,p = 1,1,1,1,1,1,1,1,1,1

53 ns ± 0.472 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


W Pythonie 3.8 już nie widać tutaj różnicy. We wcześniejszych wersjach mogła być dostrzegalna.

##### Przestrzenie nazw

Poruszanie się po przestrzeniach nazw też może mieć wpływ na wydajność:

In [156]:
def kwadraty_v1(lista):
    output = []
    for element in lista:
        output.append(element**2)
    return output

In [157]:
def kwadraty_v2(lista):
    output = []
    append = output.append  # Tu dzieje się magia
    for element in lista:
        append(element**2)
    return output

In [160]:
%timeit -n10 kwadraty_v1(MILLION_ELEMENTS)

320 ms ± 8.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [161]:
%timeit -n10 kwadraty_v2(MILLION_ELEMENTS)

301 ms ± 2.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Jak widać jest trochę szybciej.. ale też jest mniej czytelnie.

### Profilowanie

Zadanie

1. Wczytaj listę filmów z pliku data/movies.txt (na początku - do testów weź pierwszych 5000 filmów, gdy kod będzie działąć szybko dodaj kolejne)
2. Zwróć listę tytułów występujących więcej niż raz
3. Wyszukiwanie powinno być nieczułe na wielkość liter


Do tego zadania potrzebna nam będzie umiejetność korzystania z profilera. To taki specjalny program, który pozwala nam oszacować różne parametry związane z wykonywaniem naszych programów - np. czas w jakim się wykonuje całość i poszczególne funkcje w ramach programu

Wykorzystamy do tego dekorator

In [307]:
import cProfile, pstats, io

def profile(func):
    """A decorator that uses cProfile to profile a function"""
    
    def wrapper(*args, **kwargs):
        pr = cProfile.Profile()
        pr.enable()
        retval = func(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = "cumulative"
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval
    return wrapper

In [309]:
import time

@profile
def list_to_str(lista):
    output = []
    for el in lista:
        output.append(str(el))
        time.sleep(0.001)
    return "\n".join(output)

list_to_str(range(1, 5000))

         10001 function calls in 5.618 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.054    0.054    5.618    5.618 <ipython-input-309-7a698e362e27>:3(list_to_str)
     4999    5.556    0.001    5.556    0.001 {built-in method time.sleep}
     4999    0.008    0.000    0.008    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





'1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25\n26\n27\n28\n29\n30\n31\n32\n33\n34\n35\n36\n37\n38\n39\n40\n41\n42\n43\n44\n45\n46\n47\n48\n49\n50\n51\n52\n53\n54\n55\n56\n57\n58\n59\n60\n61\n62\n63\n64\n65\n66\n67\n68\n69\n70\n71\n72\n73\n74\n75\n76\n77\n78\n79\n80\n81\n82\n83\n84\n85\n86\n87\n88\n89\n90\n91\n92\n93\n94\n95\n96\n97\n98\n99\n100\n101\n102\n103\n104\n105\n106\n107\n108\n109\n110\n111\n112\n113\n114\n115\n116\n117\n118\n119\n120\n121\n122\n123\n124\n125\n126\n127\n128\n129\n130\n131\n132\n133\n134\n135\n136\n137\n138\n139\n140\n141\n142\n143\n144\n145\n146\n147\n148\n149\n150\n151\n152\n153\n154\n155\n156\n157\n158\n159\n160\n161\n162\n163\n164\n165\n166\n167\n168\n169\n170\n171\n172\n173\n174\n175\n176\n177\n178\n179\n180\n181\n182\n183\n184\n185\n186\n187\n188\n189\n190\n191\n192\n193\n194\n195\n196\n197\n198\n199\n200\n201\n202\n203\n204\n205\n206\n207\n208\n209\n210\n211\n212\n213\n214\n215\n216\n217\n218\n219\n220\n221\n22

Widzimy ile razy dany fragment był wywołany (`ncalls`) i w odpowiednich kolumnach widzimy ile to zajęło czasu. 

Spróbujcie teraz napisać rozwiązanie zadania, sprofiluj je a potem zoptymalizuj


- Wczytaj listę filmów z pliku data/movies.txt (na początku - do testów weź pierwszych 5000 filmów, gdy kod będzie działąć szybko dodaj kolejne)
- Zwróć listę tytułów występujących więcej niż raz
- Wyszukiwanie powinno być nieczułe na wielkość liter


In [205]:


# ver 0
def read_movies(src, N=5000):
    with open(src) as fh:
        return fh.read().splitlines()[:N]

def is_duplicate(needle, haystack):

    for movie in haystack:
        if needle.lower() == movie.lower():
            return True
    return False


def find_duplicate_movies(src="data/movies.csv"):
    movies = read_movies(src)
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate(movie, movies):
            duplicates.append(movie)
    return duplicates


Zmierzmy czas wykonania takiego wyszukiwania:


In [206]:
%timeit -n1 find_duplicate_movies()

2.21 s ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Postarajmy się sprofilować nasz kod i zobaczyć co tam się dzieje.
W tym celu stworzymy dekorator `@profile`, który umożliwi profilowanie funkcji i wypisze raport. Napiszemy to w oparciu o: https://docs.python.org/3/library/profile.html#profile.Profile

In [208]:
import cProfile, pstats, io

def profile(func):
    """A decorator that uses cProfile to profile a function"""
    
    def wrapper(*args, **kwargs):
        pr = cProfile.Profile()
        pr.enable()
        retval = func(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = "cumulative"
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval
    return wrapper

Udekorujemy teraz funkcję, którą chcemy profilować. Można to zrobić w oryginalnej celce, ale zachowam tutaj kolejność i przepisze kod jeszcze raz

In [224]:
# ver 1 - dodany dekorator profilera

def read_movies(src, N=5000):
    with open(src) as fh:
        return fh.read().splitlines()[:N]

def is_duplicate(needle, haystack):

    for movie in haystack:
        if needle.lower() == movie.lower():
            return True
    return False

@profile
def find_duplicate_movies(src="data/movies.csv"):
    movies = read_movies(src)
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate(movie, movies):
            duplicates.append(movie)
    return duplicates

find_duplicate_movies()

         24690878 function calls in 5.485 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    5.485    5.485 <ipython-input-224-640e1aa00b9e>:14(find_duplicate_movies)
     5000    3.165    0.001    5.395    0.001 <ipython-input-224-640e1aa00b9e>:7(is_duplicate)
 24679772    2.231    0.000    2.231    0.000 {method 'lower' of 'str' objects}
        1    0.008    0.008    0.084    0.084 <ipython-input-224-640e1aa00b9e>:3(read_movies)
        1    0.054    0.054    0.054    0.054 {method 'splitlines' of 'str' objects}
        1    0.005    0.005    0.021    0.021 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.015    0.015 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
        1    0.015    0.015    0.015    0.015 {built-in method _codecs.utf_8_decode}
     5000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
    

['Great Buster: A Celebration',
 'Grace Is Gone',
 'God Inside My Ear',
 'Get Big',
 'Frank And Ava',
 'Forrest Gump',
 'For Love Or Money',
 'Farinelli',
 'Fantomas Three Film Collection: Fantomas / Fantomas Unleashed / Fantomas Vs. Scotland Yard',
 'face in the crowd',
 'F1 2018 Official Review',
 'Egg',
 'Dragon Unleashed',
 'Dragged Across Concrete',
 "Dog's Way Home",
 'Divorce Party',
 'Coyotaje',
 'Cold Sweat',
 'shame',
 'Electric Love',
 "Dr. Seuss' The Grinch",
 "Dr. Seuss' The Grinch",
 "Dr. Seuss' The Grinch",
 'Power',
 'Power',
 'Outlander',
 'Zizou And The Arab Spring',
 'Wonder Park',
 'Wonder Dogs',
 'White Chamber',
 'What Men Want',
 'Vice',
 'Triple Threat',
 'Trickster',
 'Trickster',
 'trading paint',
 'Upside',
 'Unity Of Heroes',
 'Rainbow',
 'Prodigy',
 'LEGO Movie 2: The Second Part',
 'Kid Who Would Be King',
 'Kid Who Would Be King',
 'Heiress',
 'great flip-off',
 'City That Sold America',
 'Stray',
 'Skin',
 'Serenity',
 'Run The Race',
 'Replicas',
 'Repl

Widzimy winowajcę:

            1    0.005    0.005    5.454    5.454 <ipython-input-209-7044ee3853c0>:12(find_duplicate_movies)
         5000    3.095    0.001    5.367    0.001 <ipython-input-209-7044ee3853c0>:5(is_duplicate)
     24679772    2.272    0.000    2.272    0.000 {method 'lower' of 'str' objects}
     
Zmieńmy więc trochę implementację:



In [225]:
# ver 2 - optymalizacja ze względu na lower()
def read_movies(src, N=5000):
    with open(src) as fh:
        return fh.read().splitlines()[:N]

def is_duplicate(needle, haystack):

    for movie in haystack:
        if needle == movie:
            return True
    return False

@profile
def find_duplicate_movies(src="data/movies.csv"):
    movies = [movie.lower() for movie in read_movies(src)]
    duplicates = []
    while movies:
        movie = movies.pop()
        if is_duplicate(movie, movies):
            duplicates.append(movie)
    return duplicates

find_duplicate_movies()

         16107 function calls in 0.517 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    0.517    0.517 <ipython-input-225-f25c11067c6f>:13(find_duplicate_movies)
     5000    0.412    0.000    0.412    0.000 <ipython-input-225-f25c11067c6f>:6(is_duplicate)
        1    0.011    0.011    0.098    0.098 <ipython-input-225-f25c11067c6f>:2(read_movies)
        1    0.072    0.072    0.072    0.072 {method 'splitlines' of 'str' objects}
        1    0.006    0.006    0.015    0.015 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.009    0.009 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
        1    0.009    0.009    0.009    0.009 {built-in method _codecs.utf_8_decode}
        1    0.002    0.002    0.003    0.003 <ipython-input-225-f25c11067c6f>:15(<listcomp>)
     5000    0.001    0.000    0.001    0.000 {method 'lower' of 'str' o

['great buster: a celebration',
 'grace is gone',
 'god inside my ear',
 'get big',
 'frank and ava',
 'forrest gump',
 'for love or money',
 'farinelli',
 'fantomas three film collection: fantomas / fantomas unleashed / fantomas vs. scotland yard',
 'face in the crowd',
 'f1 2018 official review',
 'egg',
 'dragon unleashed',
 'dragged across concrete',
 "dog's way home",
 'divorce party',
 'coyotaje',
 'cold sweat',
 'shame',
 'electric love',
 "dr. seuss' the grinch",
 "dr. seuss' the grinch",
 "dr. seuss' the grinch",
 'power',
 'power',
 'outlander',
 'zizou and the arab spring',
 'wonder park',
 'wonder dogs',
 'white chamber',
 'what men want',
 'vice',
 'triple threat',
 'trickster',
 'trickster',
 'trading paint',
 'upside',
 'unity of heroes',
 'rainbow',
 'prodigy',
 'lego movie 2: the second part',
 'kid who would be king',
 'kid who would be king',
 'heiress',
 'great flip-off',
 'city that sold america',
 'stray',
 'skin',
 'serenity',
 'run the race',
 'replicas',
 'repl

Czy potrzebujemy teraz funkcji is_duplicate? Zamiast pętli moglibyśmy sprawdzić przecież przynależność

In [226]:
# ver 3 sprawdzenie przynależności zamiast pętli
def read_movies(src, N=5000):
    with open(src) as fh:
        return fh.read().splitlines()[:N]

@profile
def find_duplicate_movies(src="data/movies.csv"):
    movies = [movie.lower() for movie in read_movies(src)]
    duplicates = []
    while movies:
        movie = movies.pop()
        if movie in movies:
            duplicates.append(movie)
    return duplicates

find_duplicate_movies()

         11107 function calls in 0.287 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.180    0.180    0.287    0.287 <ipython-input-226-35539f456116>:6(find_duplicate_movies)
        1    0.012    0.012    0.103    0.103 <ipython-input-226-35539f456116>:2(read_movies)
        1    0.070    0.070    0.070    0.070 {method 'splitlines' of 'str' objects}
        1    0.009    0.009    0.021    0.021 {method 'read' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.012    0.012 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
        1    0.012    0.012    0.012    0.012 {built-in method _codecs.utf_8_decode}
        1    0.001    0.001    0.003    0.003 <ipython-input-226-35539f456116>:8(<listcomp>)
     5000    0.001    0.000    0.001    0.000 {method 'lower' of 'str' objects}
     5000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
     1094 

['great buster: a celebration',
 'grace is gone',
 'god inside my ear',
 'get big',
 'frank and ava',
 'forrest gump',
 'for love or money',
 'farinelli',
 'fantomas three film collection: fantomas / fantomas unleashed / fantomas vs. scotland yard',
 'face in the crowd',
 'f1 2018 official review',
 'egg',
 'dragon unleashed',
 'dragged across concrete',
 "dog's way home",
 'divorce party',
 'coyotaje',
 'cold sweat',
 'shame',
 'electric love',
 "dr. seuss' the grinch",
 "dr. seuss' the grinch",
 "dr. seuss' the grinch",
 'power',
 'power',
 'outlander',
 'zizou and the arab spring',
 'wonder park',
 'wonder dogs',
 'white chamber',
 'what men want',
 'vice',
 'triple threat',
 'trickster',
 'trickster',
 'trading paint',
 'upside',
 'unity of heroes',
 'rainbow',
 'prodigy',
 'lego movie 2: the second part',
 'kid who would be king',
 'kid who would be king',
 'heiress',
 'great flip-off',
 'city that sold america',
 'stray',
 'skin',
 'serenity',
 'run the race',
 'replicas',
 'repl

Dopracujmy metodę read_movies - to się powinno dać zrobić szybciej. Może coś zamiast read?

In [250]:
# ver 4 optymalizacja read movies
def read_movies(src, N=5000):
    with open(src) as fh:
        return [next(fh) for i in range(N)]

@profile
def find_duplicate_movies(src="data/movies.csv"):
    movies = [movie.lower() for movie in read_movies(src)]
    duplicates = []
    while movies:
        movie = movies.pop()
        if movie in movies:
            duplicates.append(movie)
    return duplicates

find_duplicate_movies()

         16136 function calls in 0.189 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.174    0.174    0.189    0.189 <ipython-input-250-7845c6c6f76e>:6(find_duplicate_movies)
        1    0.008    0.008    0.009    0.009 <ipython-input-250-7845c6c6f76e>:8(<listcomp>)
        1    0.000    0.000    0.005    0.005 <ipython-input-250-7845c6c6f76e>:2(read_movies)
        1    0.001    0.001    0.004    0.004 <ipython-input-250-7845c6c6f76e>:4(<listcomp>)
     5000    0.003    0.000    0.003    0.000 {built-in method builtins.next}
     5000    0.001    0.000    0.001    0.000 {method 'lower' of 'str' objects}
     5000    0.001    0.000    0.001    0.000 {method 'pop' of 'list' objects}
        1    0.001    0.001    0.001    0.001 {built-in method io.open}
     1094    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
       16    0.000    0.000    0.000    0.000 /home/rkorzen/Envs/python_e

['great buster: a celebration\n',
 'grace is gone\n',
 'god inside my ear\n',
 'get big\n',
 'frank and ava\n',
 'forrest gump\n',
 'for love or money\n',
 'farinelli\n',
 'fantomas three film collection: fantomas / fantomas unleashed / fantomas vs. scotland yard\n',
 'face in the crowd\n',
 'f1 2018 official review\n',
 'egg\n',
 'dragon unleashed\n',
 'dragged across concrete\n',
 "dog's way home\n",
 'divorce party\n',
 'coyotaje\n',
 'cold sweat\n',
 'shame\n',
 'electric love\n',
 "dr. seuss' the grinch\n",
 "dr. seuss' the grinch\n",
 "dr. seuss' the grinch\n",
 'power\n',
 'power\n',
 'outlander\n',
 'zizou and the arab spring\n',
 'wonder park\n',
 'wonder dogs\n',
 'white chamber\n',
 'what men want\n',
 'vice\n',
 'triple threat\n',
 'trickster\n',
 'trickster\n',
 'trading paint\n',
 'upside\n',
 'unity of heroes\n',
 'rainbow\n',
 'prodigy\n',
 'lego movie 2: the second part\n',
 'kid who would be king\n',
 'kid who would be king\n',
 'heiress\n',
 'great flip-off\n',
 'cit

Do zrobienia jest jeszcze trochę w tym miejscu:
    
    duplicates = []
    while movies:
        movie = movies.pop()
        if movie in movies:
            duplicates.append(movie)


Wielokrotnie iterujemy po liście. Może coś tutaj dałoby się wymyśleć. 

gdybyśmy mieli taką listę: 
    
    [1, 3, 1, 4, 2, 3, 1]

i gdybyśmy ją posortowali 

    [1, 1, 1, 2, 3, 3, 4]

jeśli teraz zrobimy sobie dwie listy, które są wycinkami tej listy - bez pierwszego i bez ostatniego

    [1, 1, 2, 3, 3, 4]
    [1, 1, 1, 2, 3, 3]
    
Dzięki posortowaniu wiemy, że te same elementy leżeć będą obok siebie. Tutaj więc będą się zgadzać ich pozycje. Wykorzystajmy to


In [254]:
# ver 5 optymalizacja read movies
def read_movies(src, N=5000):
    with open(src) as fh:
        return [next(fh) for i in range(N)]

@profile    
def find_duplicate_movies(src="data/movies.csv"):
    movies = [movie.lower() for movie in read_movies(src)]
    movies.sort()
    duplicates = [movie1 for movie1, movie2 in zip(movies[:-1], movies[1:]) if movie1 == movie2]
    return duplicates

find_duplicate_movies()

         10044 function calls in 0.020 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.020    0.020 <ipython-input-254-027b7a8e1e20>:6(find_duplicate_movies)
        1    0.010    0.010    0.010    0.010 {method 'sort' of 'list' objects}
        1    0.003    0.003    0.004    0.004 <ipython-input-254-027b7a8e1e20>:8(<listcomp>)
        1    0.000    0.000    0.004    0.004 <ipython-input-254-027b7a8e1e20>:2(read_movies)
        1    0.002    0.002    0.004    0.004 <ipython-input-254-027b7a8e1e20>:4(<listcomp>)
     5000    0.002    0.000    0.002    0.000 {built-in method builtins.next}
     5000    0.001    0.000    0.001    0.000 {method 'lower' of 'str' objects}
        1    0.001    0.001    0.001    0.001 <ipython-input-254-027b7a8e1e20>:10(<listcomp>)
       16    0.000    0.000    0.000    0.000 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
       

['"4 months, 3 weeks and 2 days"\n',
 '"berlin, i love you"\n',
 '"black clover: season 1, part 3"\n',
 '"depatie-freleng collection, vol. 2: crazylegs crane / sheriff hoot kloot / misterjaw / the blue racer / the dogfather"\n',
 '"don\'t worry, he won\'t get far on foot"\n',
 '"fantasy makers: tolkien, lewis and macdonald"\n',
 '"frantz fanon: black skin, white mask"\n',
 '"gumby: the new adventures of gumby: 80\'s series, vol. 1"\n',
 '"hunter x hunter, vol. 5"\n',
 '"jeannette, the childhood of joan of arc"\n',
 '"juliet, naked"\n',
 '"mamie van doren film noir collection: vice raid / the girl in black stockings / guns, girls and gangsters"\n',
 '"minute to pray, a second to die"\n',
 '"my hero academia: season 3, part 1"\n',
 '"my hero academia: season 3, part 1"\n',
 '"no game no, life zero"\n',
 '"no game no, life zero"\n',
 '"nowhere, michigan"\n',
 '"one sings, the other doesn\'t"\n',
 '"pink panther cartoon collection, vol. 3: 1968 - 1969"\n',
 '"pink panther cartoon collectio

Musimy się jeszcze uodpornić na wczytanie większej liczby linii



In [260]:
# ver 6 wczytywanie większej liczby linii
def read_movies(src, N):
    with open(src) as fh:
        return [next(fh) for i in range(N)]

@profile    
def find_duplicate_movies(src="data/movies.csv", N=5000):
    movies = [movie.lower() for movie in read_movies(src, N)]
    movies.sort()
    duplicates = [movie1 for movie1, movie2 in zip(movies[:-1], movies[1:]) if movie1 == movie2]
    return duplicates

find_duplicate_movies(N=500000)

StopIteration: 

By poradzić sobie ze StopIteration możemy sięgnąć np po `itertools.islice`

In [283]:
# ver 7
import itertools

def read_movies(src, N=5000):
    with open(src) as fh:
        return list(itertools.islice(fh, N))

@profile    
def find_duplicate_movies(src="data/movies.csv", N=5000):
    movies = [movie.lower() for movie in read_movies(src, N)]
    movies.sort()
    duplicates = [movie1 for movie1, movie2 in zip(movies[:-1], movies[1:]) if movie1 == movie2]
    return duplicates

find_duplicate_movies(N=500000)

         315817 function calls in 0.281 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.281    0.281 <ipython-input-283-0677994e7e9d>:8(find_duplicate_movies)
        1    0.078    0.078    0.133    0.133 <ipython-input-283-0677994e7e9d>:10(<listcomp>)
   313240    0.055    0.000    0.055    0.000 {method 'lower' of 'str' objects}
        1    0.054    0.054    0.054    0.054 {method 'sort' of 'list' objects}
        1    0.042    0.042    0.046    0.046 <ipython-input-283-0677994e7e9d>:4(read_movies)
        1    0.034    0.034    0.034    0.034 <ipython-input-283-0677994e7e9d>:12(<listcomp>)
     1283    0.001    0.000    0.004    0.000 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
     1283    0.002    0.000    0.002    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000 

['"007: the roger moore collection, vol. 1: live and let die / man with the golden gun / spy who loved me"\n',
 '"007: the roger moore collection, vol. 2: for your eyes only / moonraker / octopussy"\n',
 '"007: the sean connery collection, vol. 1"\n',
 '"007: the sean connery collection, vol. 2"\n',
 '"1, 2, 3 soleils"\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 black men named george"\n',
 '"10,000 saints"\n',
 '"13 grandes exitos del ano, vol. 1"\n',
 '"2 took t.v., vol. 1"\n',
 '"20,000 days on earth"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the s

In [284]:
# ver 8 
import itertools

def read_movies(src, N=5000):
    with open(src) as fh:
        return list(itertools.islice(fh, N))

@profile    
def find_duplicate_movies(src="data/movies.csv", N=5000):
    l = str.lower
    movies = [l(movie) for movie in read_movies(src, N)]
    movies.sort()
    duplicates = [movie1 for movie1, movie2 in zip(movies[:-1], movies[1:]) if movie1 == movie2]
    return duplicates

find_duplicate_movies(N=500000)

         315817 function calls in 0.270 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.270    0.270 <ipython-input-284-c67c7434e24e>:8(find_duplicate_movies)
        1    0.063    0.063    0.120    0.120 <ipython-input-284-c67c7434e24e>:11(<listcomp>)
        1    0.059    0.059    0.059    0.059 {method 'sort' of 'list' objects}
   313240    0.057    0.000    0.057    0.000 {method 'lower' of 'str' objects}
        1    0.039    0.039    0.043    0.043 <ipython-input-284-c67c7434e24e>:4(read_movies)
        1    0.034    0.034    0.034    0.034 <ipython-input-284-c67c7434e24e>:13(<listcomp>)
     1283    0.001    0.000    0.003    0.000 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
     1283    0.002    0.000    0.002    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000 

['"007: the roger moore collection, vol. 1: live and let die / man with the golden gun / spy who loved me"\n',
 '"007: the roger moore collection, vol. 2: for your eyes only / moonraker / octopussy"\n',
 '"007: the sean connery collection, vol. 1"\n',
 '"007: the sean connery collection, vol. 2"\n',
 '"1, 2, 3 soleils"\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 black men named george"\n',
 '"10,000 saints"\n',
 '"13 grandes exitos del ano, vol. 1"\n',
 '"2 took t.v., vol. 1"\n',
 '"20,000 days on earth"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the s

         315817 function calls in 0.274 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.012    0.012    0.274    0.274 <ipython-input-278-1e7856993feb>:8(find_duplicate_movies)
        1    0.064    0.064    0.118    0.118 <ipython-input-278-1e7856993feb>:10(<listcomp>)
        1    0.059    0.059    0.064    0.064 <ipython-input-278-1e7856993feb>:4(read_movies)
   313240    0.053    0.000    0.053    0.000 {method 'lower' of 'str' objects}
        1    0.051    0.051    0.051    0.051 {method 'sort' of 'list' objects}
        1    0.030    0.030    0.030    0.030 <ipython-input-278-1e7856993feb>:12(<listcomp>)
     1283    0.001    0.000    0.004    0.000 /home/rkorzen/Envs/python_efficient_course/lib/python3.7/codecs.py:319(decode)
     1283    0.002    0.000    0.002    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000 

['"007: the roger moore collection, vol. 1: live and let die / man with the golden gun / spy who loved me"\n',
 '"007: the roger moore collection, vol. 2: for your eyes only / moonraker / octopussy"\n',
 '"007: the sean connery collection, vol. 1"\n',
 '"007: the sean connery collection, vol. 2"\n',
 '"1, 2, 3 soleils"\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 b.c."\n',
 '"10,000 black men named george"\n',
 '"10,000 saints"\n',
 '"13 grandes exitos del ano, vol. 1"\n',
 '"2 took t.v., vol. 1"\n',
 '"20,000 days on earth"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the sea"\n',
 '"20,000 leagues under the s

In [276]:
def xxx(text):
    l = str.lower
    return l(text)

def xxx2(text):
    return text.lower()
    

In [269]:
%timeit xxx("QWE")

189 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [270]:
%timeit xxx2("QWE")

135 ns ± 2.82 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Podsumowując

* Są różne rodzaje optymalizacji (szybkość, pamięć, stabilność itp)
* Są różne poziomy optymalizacji (projekt, algorytmy, struktury danych, kod źródłowy)
* Optymalizacja kodu źródłowego sumuje się - każda poprawka wnosi swój wkład w przyspieszenie
* Optymalizacja kodu źródłowego jest stosunkowo prosta i "tania" - wystarczy, że będziemy trzymać się idiomów pythona, korzystać z funkcji wbudowanych, biblioteki standardowej. Nie wymyślajmy koła na nowo.
* Przed optymalizacją dokończ swój kod - spraw by powstała działająca wersja, napisz testy, które pomogą uniknąć błędów regresji, przetestuj swój kod. Zapisz wyniki i czas by mieć z czym porównywać. Profiluj swój kod. 